In [1]:
import threading
import time
import queue
from pynq.overlays.base import BaseOverlay
from random import randint
base = BaseOverlay("base.bit")

In [2]:
btns = base.btns_gpio

def blink(t, f, n):
    '''
    Function to blink the LEDs
    Params:
      t: amount of time to blink for (s)
      f: frequency at which to blink the LED (Hz)
      n: index of the LED (0 to 4) - 4 is rgb LED
    '''
    end_time = time.time() + t
    t_on = 1/(2*f)
    led_on = False
    
    if n == 4:
        '''
        for i in range(1,t+1):
            led_pwr = i % 2
            pins['green'].write(led_pwr)
            time.sleep(d)
        pins['green'].write(0)
        '''

        while time.time() < end_time:
            if not led_on:
                base.rgbleds[4].on(0b010)
                led_on = True
            else:
                base.rgbleds[4].off()
                led_on = False
            time.sleep(t_on)
        base.rgbleds[4].off()
            
    else:
        while time.time() < end_time:
            base.leds[n].toggle()
            time.sleep(t_on)
        base.leds[n].off()
    
def turn_leds_off():
    
    base.rgbleds[4].off()
    for n in range(4):
        base.leds[n].off()

In [3]:
turn_leds_off()

In [3]:
blink(3,30,4)
print(-2%5)

3


In [3]:
def philosopher_t(left_fork, right_fork, num):
    '''
    Philosopher function to try and acquire resource to eat
    left and right forks - forks (locks)
    num: index representing the LED and thread number.
    '''
    global eating, a
    
    using_left_fork = False
    using_right_fork = False
    is_next = False
    has_rtt = False
    
    #only want to take ANY forks if you can get both 
    while not stop:
        if using_left_fork:
            left_fork.release()
        if using_right_fork:
            right_fork.release()
        
        #Entering Starving state
        print("{}: Philosopher {} is starving".format(time.strftime("%H:%M:%S"),num))
        
        #enter loop to determine who is next to eat
        #want to schedule highest priority that can eat after the next one finishes
        while next_to_eat != num:
            if stop:
                break
            
        if stop:
            break
            
        while not (using_left_fork and using_right_fork):
            #has_rtt = rtt.acquire(True)
            #print("Philosopher {} has rtt".format(num))
            using_left_fork = left_fork.acquire(False)
            if using_left_fork:
                #print("Philosopher {} has left fork".format(num))
                using_right_fork = right_fork.acquire(False)
                if not using_right_fork:
                    left_fork.release()
                    #rtt.release()
                    using_left_fork = False
                    has_rtt = False
                    #print("Philosopher {} put down left fork and relinquished rtt".format(num))
                    if stop:
                        break
                    time.sleep(1)
            else:
                #rtt.release()
                has_rtt = False
                #print("Philosopher {} relinquished rtt".format(num))
                if stop:
                    break
                time.sleep(1)


        if stop:
            if using_left_fork:
                left_fork.release()
            if using_right_fork:
                right_fork.release()
            using_left_fork = False
            using_right_fork = False
            break
            
        #has both forks, entering Eating state
        #rtt.release()
        #has_rtt = False
        eating[num] = True
        a[num] = a[num] + 1
        print("{}: Philosopher {} is eating".format(time.strftime("%H:%M:%S"),num))
        blink(EAT_TIME, 25, num)
        
        #done eating
        left_fork.release()
        right_fork.release()
        eating[num] = False
        q.put(num)
        using_left_fork = False
        using_right_fork = False
        if stop:
            break
        print("{}: Philosopher {} is napping".format(time.strftime("%H:%M:%S"),num))
        blink(SLEEP_TIME, 0.5, num)
        

In [4]:
def button_t():
    global stop
    while not stop:
        if btns.read() != 0:
            stop = True
    print("Button was pressed! Ending program...")
    
def waiter_t():
    global next_to_eat, a, eating
    
    #initialization - first two philosophers
    next_to_eat = nextq.get()
    eating[next_to_eat] = True
    time.sleep(0.1) #give enough time for that thread to pass the block before running this again
    next_to_eat = nextq.get()
    eating[next_to_eat] = True
    time.sleep(0.1) #give enough time for that thread to pass the block before running this again
        
    while not stop:
        #while q.qsize() < 2: #wait for next one to begin eating to make sure the correct order is preserved
        #    if stop:
        #        break
        #if stop:
        #    break
        last_finished = q.get(True,20)
        print("Last finished: {}".format(last_finished))
        #determine priority (lower values imply hungrier)
        if next_to_eat == 0 or eating[0]:
            if next_to_eat == 1 or eating[1]:
                if next_to_eat == 2 or eating[2]:
                    min_i = 3 #don't let a philosopher eat twice in a row - will cause issues
                else:
                    min_i = 2
            else:
                min_i = 1
        else:
            min_i = 0
        for i in range(5):
            if (i == next_to_eat) or eating[i]: #don't let a philosopher eat twice in a row - will cause issues
                continue
            if a[i] < a[min_i]:
                min_i = i
        print(a[0],a[1],a[2],a[3],a[4])
        print("Highest priority: {}".format(min_i))
        #determine if that philosopher is able to eat next (forks are available now):
        if able_to_eat(min_i):
            #nextq.put(min_i)
            eating[min_i] = True #block eating now so it doesn't mess up future calculations
            next_to_eat = min_i
            print("{} is able to eat next".format(min_i))
        else:
            #if not, pick the one of these two with the highest priority:
            op1 = (last_finished - 1) % 5
            op2 = (last_finished + 1) % 5
            if a[op1] < a[op2] & able_to_eat(op1):
                #nextq.put(op1)
                eating[op1] = True
                next_to_eat = op1
                print("{} is eating next".format(op1))
            elif able_to_eat(op2):
                #nextq.put(op2)
                eating[op2] = True
                next_to_eat = op2
                print("{} is eating next".format(op2))
            elif able_to_eat(op1):
                #nextq.put(op2)
                eating[op1] = True
                next_to_eat = op1
                print("{} is eating next".format(op1))
            else:
                print("Something bad happened")
        #next_to_eat = nextq.get()
        time.sleep(0.1) #give enough time for that thread to pass the block before running this again
        
        
        

In [9]:
def able_to_eat(index):
    global eating
    '''
    if (not eating[index-1]) or (((index-1)%5) == next_to_finish):
        right_available = True
    else:
        right_available = False
        
    if (not eating[(index+1)%5]) or (((index+1)%5) == next_to_finish):
        left_available = True
    else:
        right_available = False
        
    if right_available and left_available:
        can_eat = True
    else:
        can_eat = False
    '''
        
        
    #return (not eating[index-1]) and (not eating[(index+1)%5])
    return (not eating[index-1]) and (not eating[index]) and (not eating[(index+1)%5])
            

In [9]:
# Initialize and launch the threads
threads = []
fork0 = threading.Lock()
fork1 = threading.Lock()
fork2 = threading.Lock()
fork3 = threading.Lock()
fork4 = threading.Lock()
rtt = threading.Lock() #right to take

EAT_TIME = 5
SLEEP_TIME = 10

stop = False #stop flag for button interrupt
next_to_eat = 0 #global var for scheduling next to eat

q = queue.Queue() #queue to keep track of the order the philosophers finished eating
a = [0, 0, 0, 0, 0] #array to keep track of priority (how many times the philosophers ate)
eating = [False, False, False, False, False]
nextq = queue.Queue()

#start with 0 and 2
nextq.put(0)
nextq.put(2)

for i in range(5):
    if i == 0:
        left_fork = fork0
        right_fork = fork4
    elif i == 1:
        left_fork = fork1
        right_fork = fork0
    elif i == 2:
        left_fork = fork2
        right_fork = fork1
    elif i == 3:
        left_fork = fork3
        right_fork = fork2
    elif i == 4:
        left_fork = fork4
        right_fork = fork3
        
    t = threading.Thread(target=philosopher_t, args=(left_fork, right_fork, i))
    threads.append(t)
    t.start()
    
#thread for button interrupt
t = threading.Thread(target=waiter_t, args=())
threads.append(t)
t.start()
    
#thread for button interrupt
t = threading.Thread(target=button_t, args=())
threads.append(t)
t.start()

for t in threads:
    name = t.getName()
    t.join()
    print('{} joined'.format(name))
    
print("Program terminated")

03:17:54: Philosopher 0 is starving03:17:54: Philosopher 1 is starving

03:17:54: Philosopher 0 is eating
03:17:54: Philosopher 2 is starving
03:17:54: Philosopher 3 is starving03:17:54: Philosopher 4 is starving

03:17:54: Philosopher 2 is eating
03:17:59: Philosopher 0 is nappingLast finished: 0

1 0 1 0 0
Highest priority: 1
4 is eating next
03:17:59: Philosopher 4 is eating
03:17:59: Philosopher 2 is napping
Last finished: 2
1 0 1 0 1
Highest priority: 1
1 is able to eat next
03:17:59: Philosopher 1 is eating
03:18:04: Philosopher 4 is nappingLast finished: 4

1 1 1 0 103:18:04: Philosopher 1 is napping

Highest priority: 3
3 is able to eat next
03:18:04: Philosopher 3 is eating
Last finished: 1
1 1 1 1 1
Highest priority: 0
0 is able to eat next
03:18:09: Philosopher 0 is starving
03:18:09: Philosopher 0 is eating
03:18:09: Philosopher 2 is starving
03:18:09: Philosopher 3 is nappingLast finished: 3

2 1 1 1 1
Highest priority: 1
2 is eating next
03:18:09: Philosopher 2 is eating


## Part 2.2

In [11]:
def philosopher_t(left_fork, right_fork, num):
    '''
    Philosopher function to try and acquire resource to eat
    left and right forks - forks (locks)
    num: index representing the LED and thread number.
    '''
    global eating, a
    
    using_left_fork = False
    using_right_fork = False
    is_next = False
    has_rtt = False
    
    #only want to take ANY forks if you can get both 
    while not stop:
        if using_left_fork:
            left_fork.release()
        if using_right_fork:
            right_fork.release()
        
        #Entering Starving state
        print("{}: Philosopher {} is starving".format(time.strftime("%H:%M:%S"),num))
        
        #enter loop to determine who is next to eat
        #want to schedule highest priority that can eat after the next one finishes
        while next_to_eat != num:
            if stop:
                break
            
        if stop:
            break
            
        while not (using_left_fork and using_right_fork):
            #has_rtt = rtt.acquire(True)
            #print("Philosopher {} has rtt".format(num))
            using_left_fork = left_fork.acquire(False)
            if using_left_fork:
                #print("Philosopher {} has left fork".format(num))
                using_right_fork = right_fork.acquire(False)
                if not using_right_fork:
                    left_fork.release()
                    #rtt.release()
                    using_left_fork = False
                    has_rtt = False
                    #print("Philosopher {} put down left fork and relinquished rtt".format(num))
                    if stop:
                        break
                    time.sleep(1)
            else:
                #rtt.release()
                has_rtt = False
                #print("Philosopher {} relinquished rtt".format(num))
                if stop:
                    break
                time.sleep(1)


        if stop:
            if using_left_fork:
                left_fork.release()
            if using_right_fork:
                right_fork.release()
            using_left_fork = False
            using_right_fork = False
            break
            
        #has both forks, entering Eating state
        #rtt.release()
        #has_rtt = False
        eating[num] = True
        #a[num] = a[num] + 1
        print("{}: Philosopher {} is eating".format(time.strftime("%H:%M:%S"),num))
        eat_time = randint(5,15)
        a[num] = a[num] + eat_time
        blink(eat_time, 25, num)
        
        #done eating
        left_fork.release()
        right_fork.release()
        eating[num] = False
        q.put(num)
        using_left_fork = False
        using_right_fork = False
        if stop:
            break
        print("{}: Philosopher {} is napping".format(time.strftime("%H:%M:%S"),num))
        sleep_time = randint(2,eat_time)
        blink(sleep_time, 0.5, num)

In [12]:
# Initialize and launch the threads
threads = []
fork0 = threading.Lock()
fork1 = threading.Lock()
fork2 = threading.Lock()
fork3 = threading.Lock()
fork4 = threading.Lock()
rtt = threading.Lock() #right to take

stop = False #stop flag for button interrupt
next_to_eat = 0 #global var for scheduling next to eat

q = queue.Queue() #queue to keep track of the order the philosophers finished eating
a = [0, 0, 0, 0, 0] #array to keep track of priority (how long the philosophers ate)
eating = [False, False, False, False, False]
nextq = queue.Queue()

#start with 0 and 2
nextq.put(0)
nextq.put(2)

for i in range(5):
    if i == 0:
        left_fork = fork0
        right_fork = fork4
    elif i == 1:
        left_fork = fork1
        right_fork = fork0
    elif i == 2:
        left_fork = fork2
        right_fork = fork1
    elif i == 3:
        left_fork = fork3
        right_fork = fork2
    elif i == 4:
        left_fork = fork4
        right_fork = fork3
        
    t = threading.Thread(target=philosopher_t, args=(left_fork, right_fork, i))
    threads.append(t)
    t.start()
    
#thread for button interrupt
t = threading.Thread(target=waiter_t, args=())
threads.append(t)
t.start()
    
#thread for button interrupt
t = threading.Thread(target=button_t, args=())
threads.append(t)
t.start()

for t in threads:
    name = t.getName()
    t.join()
    print('{} joined'.format(name))
    
print("Program terminated")

03:44:13: Philosopher 0 is starving
03:44:13: Philosopher 0 is eating
03:44:13: Philosopher 1 is starving03:44:13: Philosopher 2 is starving

03:44:13: Philosopher 3 is starving
03:44:13: Philosopher 4 is starving
03:44:13: Philosopher 2 is eating
03:44:20: Philosopher 2 is nappingLast finished: 2
10 0 6 0 0
Highest priority: 1
3 is eating next
03:44:20: Philosopher 3 is eating

03:44:22: Philosopher 2 is starving
03:44:23: Philosopher 0 is nappingLast finished: 0

10 0 6 7 0
Highest priority: 1
1 is able to eat next
03:44:23: Philosopher 1 is eating
03:44:27: Philosopher 3 is nappingLast finished: 3
10 13 6 7 0
Highest priority: 4
4 is able to eat next

03:44:27: Philosopher 4 is eating
03:44:28: Philosopher 0 is starving
03:44:33: Philosopher 4 is napping
Last finished: 4
10 13 6 7 6
Highest priority: 2
3 is eating next
03:44:34: Philosopher 3 is starving
03:44:34: Philosopher 3 is eating
03:44:36: Philosopher 1 is nappingLast finished: 1

10 13 6 21 6
Highest priority: 2
0 is eating